Normalização do modelo BCS não linear

In [175]:
import matplotlib.pyplot as plt
#import tensorflow as tf
import scipy as sp
from scipy import signal
import numpy as np
from subrotinas import *
#from envelope import *
exec(compile(open('param.py', "rb").read(), 'param.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
%matplotlib tk

Dados carregados


### Variáveis de entrada
f $\in$ (35,65) Hz <br>
zc $\in$ (0,100)% <br>
pm $\in$ ($2\cdot 10^6$) <br>
pr

### Variáveis de estado
pbh $\in$ ($10^5, 8.5 \cdot 10^6$) <br>
pwh $\in$ ($2 \cdot 10^6, 5.2\cdot10^6$) <br>
q $\in (10^{-2},3\cdot 10^{⁻2})$ <br>



Definindo os fatores de ajuste de escala a partir dos limites operacionais

In [176]:
# Valores máximos e mínimos para normalização
#Entradas - conforme binder e pavlov
def Lim_c(x):
    return x[1]-x[0]
f_lim=(30,75)
zclim=(0,1)
pmlim=(1e5,50e5)
pbhlim=(1e5,pr) 
pwhlim=(1e5,50e5) 
qlim=(15/3600,65/3600)
pbc=Lim_c(pbhlim)
pwc=Lim_c(pwhlim)
qc=Lim_c(qlim)
pbmin=pbhlim[0]
pwmin=pwhlim[0]
qmin=qlim[0]
H_lim=(-136.31543417849096, 1420.7697113483912)
qch_lim=(0.0, 0.03290348005910621)



In [177]:
pm=20e5; #Simplificando pm fixo
#pbh  - pressão na bomba
#pwh - Pressão de fundo do poço,
#q - vazão
#PI índice de produtividade do poço
#PinC  pressão na choke
# Pressão de manifold

Definindo as variáveis simbólicas

In [178]:
# Criando simbolica
from casadi import *
nx = 3; nu = 4;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena
pbh = x[0]
pwh = x[1]
q = x[2]
#fq = x[3]; zc = x[4]

In [179]:
# Entradas

fq = u[0]; zc = u[1]; pm=u[2]; pr=u[3]
#pm=2e6;
# zc=zcref 
# fq=fqref

In [180]:
fq

MX(u[0])

In [181]:
# Calculo do HEAD e delta de press�o
q0 = ((q*qc+qmin).printme(0) / Cq * (f0 / fq))
H0 = -1.2454e6 * q0 ** 2 + 7.4959e3 * q0 + 9.5970e2
H = (CH * H0 * (fq.printme(1) / f0) ** 2)  # Head
#Pp = rho * g * H  # Delta de press�o

# Calculo da Potencia e corrente da bomba
P0 = -2.3599e9 * q0 ** 3 - 1.8082e7 * q0 ** 2 + 4.3346e6 * q0 + 9.4355e4
P = Cp * P0 * (fq / f0) ** 3;  # Potencia
I = Inp * P / Pnp  # Corrente

# Calculo da press�o de intake
F1 = (0.158 * ((rho * L1 * ((q*qc+qmin)) ** 2) / (D1 * A1 ** 2)) * (mu / (rho * D1 * ((q*qc+qmin)))) ** (1 / 4))
F2 = 0.158 * ((rho * L2 * ((q*qc+qmin)) ** 2) / (D2 * A2 ** 2)) * (mu / (rho * D2 * ((q*qc+qmin)))) ** (1 / 4)
F1.printme(0)
F2.printme(1)
pin = pbh*pbc+pbmin - rho * g * h1 - F1
# Vazao do reservatorio e vazao na choke
qr = PI * (pr - (pbh*pbc+pbmin))
qch = (zc/100)*Cc * sqrt(fabs(pwh*pwc+pwmin - pm));

# Termos não lineares
#menor q implica em menor F
funcH=Function('funcH',[x,u],[H])
funcF1=Function('funcF1',[x],[F1])
funcF2=Function('funcF2',[x],[F2])
F1lim=(funcF1([0,0,qlim[0]]),funcF1([0,0,qlim[1]]))
F2lim=(funcF2([0,0,qlim[0]]),funcF2([0,0,qlim[1]]))
F1c=Lim_c(F1lim)
F2c=Lim_c(F2lim)
Hc=Lim_c(H_lim)
qcc=Lim_c(qch_lim)
print('Limites: pbh,pwh,q')
print(pbhlim,pwhlim,qlim)
print('Limites: F1,F2,H,qch')
print(F1lim,F2lim,H_lim,qch_lim)


Limites: pbh,pwh,q
(100000.0, 12600000.0) (100000.0, 5000000.0) (0.004166666666666667, 0.018055555555555554)
Limites: F1,F2,H,qch
(DM(99811.5), DM(107923)) (DM(239548), DM(259016)) (-136.31543417849096, 1420.7697113483912) (0.0, 0.03290348005910621)


In [182]:
# F1c=941799.5331
# F2c=2260318.8795
#qcc=0.033987702
#Hc=1511.97
#Normalizar termos não lineares
##########################
qch=(qch-qch_lim[0])/qcc
F1=((F1-F1lim[0])/F1c)
F2=((F2-F2lim[0])/F2c)
H=(H-H_lim[0])/Hc
###########################

#pr*b1*PI/V1
# qch=qch*qcc
# F1=F1c*F1
# F2=F2c*F2
# H=Hc*H
dpbhdt = (1/pbc)*b1/V1*(qr - (q*qc+qmin))
dpwhdt = (1/pwc)*b2/V2*((q*qc+qmin) - (qcc*qch+qch_lim[0]))
dqdt = (1/(qc*M))*(pbh*pbc+pbmin - (pwh*pwc+pwmin) - rho*g*hw - (F1c*F1+F1lim[0]) - (F2c*F2+F2lim[0]) + rho * g * (H*Hc+H_lim[0]))
# dfqdt = (fqref - fq)/tp[0];
# dzcdt = (zcref - zc)/tp[1];

dxdt = vertcat(dpbhdt,dpwhdt,dqdt);

# Restricao do Elemento Final
#dudt = vertcat(if_else(fabs(dfqdt)>dudt_max[0],sign(dfqdt)*dudt_max[0],dfqdt),
#       if_else(fabs(dzcdt)>dudt_max[1],sign(dzcdt)*dudt_max[1],dzcdt));

In [183]:
xss = np.float32(np.array([8311024.82175957,2990109.06207437,0.00995042241351780]))
x0=np.array([pbmin,pwmin,qmin])
xc=np.array([pbc,pwc,qc])
xssn = (xss-x0)/xc


In [184]:

xssn

array([0.656882  , 0.58981816, 0.41643043])

In [185]:
uss = np.array([50,50,20e5,1.26e7])
funcx1dot=Function('funcx1dot',[u,x],[dxdt[0]])
funcx2dot=Function('funcx2dot',[u,x],[dxdt[1]])
funcx3dot=Function('funcx3dot',[u,x],[dxdt[2]])
dx=[funcx1dot(uss,xssn), funcx2dot(uss,xssn),funcx3dot(uss,xssn)]

dx

|> 0 : 9.9504226818680763e-03
|> 1 : 5.0000000000000000e+01


[DM(-2.01836e-08), DM(1.82581e-08), DM(4.15065e-08)]

In [186]:
xss

array([8.311025e+06, 2.990109e+06, 9.950423e-03], dtype=float32)

In [187]:
# Função casadi
#dxdt = casadi.vertcat(dpbhdt,dpwhdt,dqdt) 
Eq_Estado = casadi.Function('Eq_Estado',[x,u],[vertcat(dpbhdt,dpwhdt,dqdt)],
                     ['x','u'],['dxdt'])

y=vertcat(pin,H);
ny = y.size1()
# Equações algébricas
sea_nl = casadi.Function('sea_nl',[x,u],[y,pin,H],\
                 ['x','u'],['y','pin','H']); # Sistema de Eq. Algebricas variaveis de sa�da

BCS={
     'x': x,
     'u': u,
     'y': y,
     'nx': nx,
     'nu': nu,
     'ny': ny,
     'NaoLinear': {'sedo_nl': Eq_Estado(x,u),
                   'sea_nl': sea_nl
                   }
}
#%% Calculo do estacionario
#% Func��o objetivo
dxdt_0 = Eq_Estado(BCS['x'], BCS['u']);
J = sum1(dxdt_0**2);
#% Otimizador
opt={
     'ipopt':{
         'print_level':0,
         'acceptable_tol':1e-8,
         'acceptable_obj_change_tol':1e-6,
         'max_iter':50
         },
     'print_time':0,
     }

opt['ipopt']['print_level']=0;# %0,3
opt['print_time']=0;
opt['ipopt']['acceptable_tol']=1e-8;
opt['ipopt']['acceptable_obj_change_tol']=1e-6;
opt['ipopt']['max_iter']=50;



In [192]:
res=Eq_Estado(np.array([0.656882,0.58981816,0.41643043]),uss)
print(res)

|> 0 : 9.9504226388888884e-03
|> 1 : 5.0000000000000000e+01
[-1.89114e-08, 1.94355e-08, 5.45536e-08]


In [188]:
MMQ = {'x':BCS['x'], 'f':J, 'p':BCS['u']}
#nlp={'x':vertcat(BCS['x'],BCS['u']), 'f':J} #variáveis de decisão, função f, g (N/A)
#nlp={'x':BCS['x'], 'f':J}
solver = nlpsol('solver', 'ipopt', MMQ, opt)
# Restrições das variaveis de decis�o
# minimo
args={
      'lbx': np.zeros((nx,1)),
# m�ximo
      'ubx':np.full((nx, 1), np.inf)
      }

# Solu��o do otimizador
sol=solver(x0=BCS['x'], lbx=args['lbx'], ubx=args['ubx'], p=BCS['u']);
yss=sea_nl(sol['x'],BCS['u'])
Estacionario = Function('Estacionario',[BCS['x'],BCS['u']],\
    [sol['x'],yss[0]],\
    ['x0','uss'],['xss','yss']);

BCS['Estacionario'] = Estacionario;
f_ss,zc_ss,pm_ss,pr_ss= (np.array([50, 50,20e5,1.26e7]))
uss = np.array([f_ss,zc_ss,pm_ss,pr_ss]); # Entradas do estacionario
# uss_n=normalizar_u(uss,unorm)
# uss_n

sol=solver(x0=xssn, p=uss);

print(sol['x']*xc+x0, xss)
#sol['x']*xc



|> 0 : 9.9504226818680763e-03
|> 1 : 5.0000000000000000e+01
|> 0 : 9.9504226818680763e-03
|> 1 : 5.0000000000000000e+01
|> 0 : 9.9504226818680763e-03
|> 1 : 5.0000000000000000e+01
|> 1 : 5.0000000000000000e+01
|> 0 : 9.9504226818680763e-03
|> 0 : 9.9504224135177587e-03
|> 1 : 5.0000000000000000e+01
|> 0 : 9.9504224135177587e-03
|> 1 : 5.0000000000000000e+01
|> 0 : 9.9504224135177587e-03
|> 1 : 5.0000000000000000e+01
[8.31102e+06, 2.99011e+06, 0.00995042] [8.311025e+06 2.990109e+06 9.950423e-03]


In [191]:
xc,x0

(array([1.25000000e+07, 4.90000000e+06, 1.38888889e-02]),
 array([1.00000000e+05, 1.00000000e+05, 4.16666667e-03]))

In [189]:
c.

SyntaxError: invalid syntax (1195783164.py, line 1)

In [ ]:
sol=solver(x0=xssn, p=uss);
print(sol['x']*xc+x0)
sol=solver(x0=xssn, p=uss);
print(sol['x']*xc+x0)

|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698193625756196e+05
|> 0 : 4.4698193625756196e+05
|> 0 : 4.4698193625756196e+05
[8.31102e+06, 2.99011e+06, 0.00995042]
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698193625756196e+05
|> 0 : 4.4698193625756196e+05
|> 0 : 4.4698193625756196e+05
[8.31102e+06, 2.99011e+06, 0.00995042]


In [ ]:
# Definir variaveis manipuladas e controladas e disturbio externo
mv = [0,1]    #% [f, Zc]
pv = [0,1]  #% [pin, H]  #% [P, I]
#pv = [2,3];  #% [pin, H]
de = [0];      #% [pm]
tg = 2;      #% MV target
#% Parametros
ts = 1;
#%Modelo de predição
#% Criando o objeto para predição do modelo
# Iniciando variavel dicionário para a construção da EDO
# sedo = {'x': BCS['x'][0:3], # Estados
#         'p': BCS['u'], #Variáveis exogenas
#         'ode': BCS['NaoLinear']['sedo_nl'] # SEDO (Gerado no bcs_settings)
#         };

sedo = {'x': BCS['x'], # Estados
        'p': BCS['u'], #Variáveis exogenas
        'ode': Eq_Estado(x,u) # SEDO (Gerado no bcs_settings)
        };

#% Criando o objeto p,ra integração da Eq_estado
opt = {'tf':ts,
       't0':0

       };   #% opções do integrador

In [ ]:
int_odes = integrator('int_odes','cvodes',sedo,opt)
# objeto integrador
res = int_odes(x0=BCS['x'],p=BCS['u']);             #   % solução um passo a frente
npv = len(pv); nmv = len(mv); nde=len(de)

In [ ]:
# Criando o objeto para solução da equação de medição
Eq_medicao = Function('Eq_medicao',[BCS['x'],BCS['u']],[BCS['y'][pv]],['x','u'],['y']);
# Criacao do objeto para simulacao do BCS Eq de estado + Eq de Medicao
Modelo_Predicao = Function('Modelo_Predicao',[BCS['x'],BCS['u']],[res['xf'],Eq_medicao(res['xf'],BCS['u'])],['xk_1','uk_1'],['xk','yk']);
Modelo_Predicao2 = Function('Modelo_Predicao2',[BCS['x'],BCS['u']],[res['xf']],['xk_1','uk_1'],['xk']);

In [ ]:
xpk = xss;
uss
#xpk=normalizar(xss,xnorm)

#uk_1 = normaliza_u(uss[mv],unorm)
#uk_1=(uss[mv]-unorm[:,0])/unorm[:,1]
# MVS
# Aloca��o de variaveis
Xk = np.zeros((nx,1))

In [ ]:
#Valores iniciais de simulação
tss=5 # Tempo inicial no estacionário

tsim = 50*60; 
nsim=int(round(tsim/ts)+1)
xss_n=xssn

Yk = np.zeros((npv,1))
Uk = np.zeros((nmv+nde,1))
Ymk = Yk
Ys = Yk
Ymin = Yk
Ymax = Yk

# Povoar vetor de X com dados não normalizados
def lpf(x, omega_c, T):
    """Implement a first-order low-pass filter. 
    The input data is x, the filter's cutoff frequency is omega_c 
    [rad/s] and the sample time is T [s].  The output is y.
    """
    N=x.shape[0]
    y = x
    alpha = (2-T*omega_c)/(2+T*omega_c)
    beta = T*omega_c/(2+T*omega_c)
    for k in range(1, N):
        y[k] = alpha*y[k-1] + beta*(x[k]+x[k-1])
    return y

def step_function(tsim):
    x=np.arange(0,tsim)
    val_pm=[20e5,28e5,14e5,20e5]
    val_z=[50,100,75,95]
    val_f=[50,63,55,60]
    val_pr=[1.2e7,1e7,0.9e7]
    list_f=[(x >= 0) & (x<350), (x >= 350) & (x < 1500),(x >= 1500) & (x < 2600), x>=2600]
    list_z=[(x >= 0) & (x<700), (x >= 700) & (x < 1200), (x >= 1200) & (x < 2300),x>=2300]
    list_pm=[(x >= 0) & (x<150),  (x >=150) & (x<1000),(x >= 1000) & (x < 1800), x>=1800]
    list_pr=[(x >= 0) & (x<500),  (x >= 500) & (x < 1500), x>=1500]
    pm=np.piecewise(x,list_pm,val_pm)
    fk=np.piecewise(x,list_f, val_f)
    zc=np.piecewise(x,list_z, val_z)
    pr_z=np.piecewise(x,list_pr, val_pr)

    # Wn=2*pi*1/25
    # Wn2=2*pi*1/100    
    Wn=2*pi*0.05
    Wn2=2*pi*0.01
    pm_f=lpf(pm,Wn,1)
    pr_f=lpf(pr_z,Wn2,1)
    return np.array([lpf(fk,Wn,1),lpf(zc,Wn2*4,1),pm_f,pr_f])

label = ['f(Hz)',"zc", "pm(bar)", "Pr(bar)"];
f,zc,pm,pr=step_function(tsim)
entradas=[f,zc,pm,pr]
fig2=plt.figure()
for i,str in enumerate(label):
    ax=fig2.add_subplot(len(label),1,i+1)
    ax.plot(entradas[i].T, label='Medição')
    ax.set_ylabel(str)
    plt.grid(True)

In [ ]:
# uk_1 = np.array([[40], [70], [pm_ss]]);

nstep = tsim

def APRBS_signals():
    f=APRBS([50,65],[60,300] ,nstep)
    z=APRBS([60,100],[60,300],nstep)
    pm=APRBS([12*1e5,20*1e5],[100,200],nstep)
    pr=APRBS([1.2e7,1.4*1e7],[200,300],nstep)
    Wn=2*pi*1/25
    Wn2=2*pi*1/100    
    return  np.array([lpf(f,Wn,1),lpf(z,Wn2*2,1),lpf(pm,Wn,1),lpf(pr,Wn2,1)])
# # u_f=np.ones_like(pm_z)*40
# # u_z=np.ones_like(pm_z)*70
# #dados = np.load('BCS_data_train_limitado_f_zc5.npz')
# # u_f=dados['f']
# # u_z=dados['zc']
# uk_1 = np.array([u_f,u_z,pm_z]);

#u_f,zc,pm_z,pr_z=step_function(tsim)
#u_f,zc,pm_z,pr_z=APRBS_signals()
uk_1=step_function(tsim)
#uk_1 = APRBS_signals()
xi=(np.arange(0,int(nsim*ts)-1,ts));
#xi=np.arange(0,10,ts)






In [ ]:
Modelo_Predicao2(xpk,uk_1[:,1])

|> 0 : 1.0315656100128006e+05
|> 0 : 1.0317210970637156e+05
|> 0 : 1.0325150601900333e+05
|> 0 : 1.0325150601900333e+05
|> 0 : 1.0320402883084117e+05
|> 0 : 4.7482301048257895e+05
|> 0 : 4.7482299293091591e+05
|> 0 : 1.5788619741324671e+06
|> 0 : 5.8280373139793193e+06
|> 0 : 1.8130503897173226e+07
|> 0 : 6.2884328480598167e+07
|> 0 : 6.2884221584150463e+07
|> 0 : 1.2914926850066699e+08
|> 0 : 2.1474261139601186e+08
|> 0 : 3.9256523505415487e+08
|> 0 : 8.7684865353614652e+08
|> 0 : 2.0577064560281067e+09
|> 0 : 2.0577094699174650e+09
|> 0 : 6.0787001789272490e+09
|> 0 : 6.0787992036569538e+09
|> 0 : 4.6487751548174982e+09
|> 0 : 4.6488041630929766e+09
|> 0 : 8.0867357314703703e+09
|> 0 : 1.2303545490014444e+10
|> 0 : 1.7250531450714729e+10
|> 0 : 2.2889733302657585e+10
|> 0 : 3.3419875981616390e+10
|> 0 : 4.5577377020983734e+10
|> 0 : 6.6939418436909996e+10
|> 0 : 9.1571449029215454e+10
|> 0 : 1.1923740730109320e+11
|> 0 : 1.4972255732203458e+11
|> 0 : 1.8282598653749432e+11
|> 0 : 2.5

DM([3.52137e+06, 2.99079e+06, 3179.04])

In [ ]:
sol=solver(x0=xssn, p=uk_1[:,0]);
xpk=sol['x']
xpk

|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.0490009375952906e+05
|> 0 : 4.0490009375952906e+05
|> 0 : 4.0490009375952906e+05
|> 0 : 4.0420050792953710e+05
|> 0 : 4.0420050792953710e+05
|> 0 : 4.0420050792953710e+05
|> 0 : 4.0419361863532109e+05
|> 0 : 4.0419361863532109e+05
|> 0 : 4.0419361863532109e+05


DM([0.628055, 0.567867, 0.376398])

In [ ]:
uk_1.shape

(4, 3000)

In [ ]:
print("Simulação iniciada")
#Calcular estacionário para entrada inicial e inicializar vetores
sol=solver(x0=xssn, p=uk_1[:,0]);
xpk=sol['x']
#Inicializar Yk
ypk=sea_nl(xpk,uss)[0]
Yk=ypk*np.array([1,Hc])+[0,H_lim[0]]
ypk[1]*Hc+H_lim[0]
#inicializar Xk
Xk=xpk*xc+x0
#inicializar Uk
print(f"Valores iniciais de entrada: {uk_1[:,0].T}")
print(f"Estacionário das saídas: {Xk[:,0]}")
Uk=uk_1[:,0:1]


#for k in range(1,10):
for k in range(1,nsim-1):
    xpk = Modelo_Predicao2(xpk,uk_1[:,k])
    Xk = hcat([Xk,xpk*xc+x0]) #desnormalizar x e preencher vetor
    ypk=sea_nl(xpk,uk_1[:,k])[0]
    Yk = hcat([Yk,ypk*np.array([1,Hc])+[0,H_lim[0]]]);
    Uk = hcat([Uk,uk_1[:,k]])

#print("Xk shape ="+str(Xk.shape))
# print("Uk shape ="+str(Uk.shape))
# print("Yk shape ="+str(Uk.shape))
# print("xi shape ="+str(xi.shape))
print("Ok.simulação concluida")
Xk[0,:].shape
t=np.arange(0,Uk[0,:].shape[1])

Simulação iniciada
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.4698195735300344e+05
|> 0 : 4.0490009375952906e+05
|> 0 : 4.0490009375952906e+05
|> 0 : 4.0490009375952906e+05
|> 0 : 4.0420050792953710e+05
|> 0 : 4.0420050792953710e+05
|> 0 : 4.0420050792953710e+05
|> 0 : 4.0419361863532109e+05
|> 0 : 4.0419361863532109e+05
|> 0 : 4.0419361863532109e+05
|> 0 : 4.0419361863532109e+05
Valores iniciais de entrada: [      50       50  2000000 12000000]
Estacionário das saídas: [7.95068e+06, 2.88255e+06, 0.00939442]
|> 0 : 4.0419361863532109e+05
|> 0 : 4.0419361863532109e+05
|> 0 : 4.0419361863531551e+05
|> 0 : 4.0419361863531551e+05
|> 0 : 4.0419361863531842e+05
|> 0 : 4.0419361860731512e+05
|> 0 : 4.0419361856755987e+05
|> 0 : 4.0419361858856329e+05
|> 0 : 4.0419361858856329e+05
|> 0 : 4.0419361858856329e+05
|> 0 : 4.0419361858856084e+05
|> 0 : 4.0419361858856084e+05
|> 0 : 4.0419361858856207e+05
|> 0 : 4.0419361857628234e+05
|> 0 : 4.0

In [ ]:
uk_1[:,0:1]

array([[      50],
       [      50],
       [ 2000000],
       [12000000]])

In [ ]:
# fig1=plt.figure()
# label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
# for iy in range(0,npv):
#     ax = fig1.add_subplot(npv,1,iy+1)
#     #print(iy)
#     if iy == 0: # Pin
#         ax.plot(xi,(Yk[iy,:].T)/1e5, label='Medição')
#         #ax.plot(xi,Ymk[iy,:].T/1e5, label='EKF')
#         ax.set_ylabel(label[iy])
#         ax.set(xlim=(xi[0], nsim*ts))
#        # ax.set(ylim=(40,62))
#         plt.grid(True)
#     else: # H
#         ax.plot(xi,Yk[iy,:].T, label='Medição')
#         #ax.plot(xi,Ymk[iy,:].T,label='EKF')
#         ax.set_ylabel(label[iy])
#         ax.set(xlim=(xi[0], nsim*ts))
#        # ax.set(ylim=(580, 850))
#         plt.grid(True)
# #ax.plot(xi,Yk[2,:].T, label='EKF')
# ax.legend();
# ax.set_xlabel('Time (nT)')
# fig1.show()


In [ ]:
# #% Restrição
umin  = np.array([35, 70]); np.transpose(umin);  # lower bounds of inputs
umax  = np.array([65, 100]); np.transpose(umax); # upper bounds of inputs
dumax = np.array([0.5, dzc_max]); np.transpose(dumax);
fig2=plt.figure()
label = ['f(Hz)',r'$z_c$(%)', "pm(bar)", "Pr(bar)"];
Uk[2:,:]=Uk[2:,:]/1e5


for i,str in enumerate(label):
   
    ax=fig2.add_subplot(len(label),1,i+1)
    ax.plot(t,Uk[i,:].T, label='Medição')
    # if i<2:
    #     ax.plot([1,nsim],[umin[i], umin[i]],'--r')
    #     ax.plot([1,nsim],[umax[i], umax[i]],'--r', label='Restrição')
    ax.set_ylabel(str)
    #ax.set(xlim=(xi[0], nsim*ts))
    # if i==0:
    #     ax.set(ylim=(30, 70))
    plt.grid(True)

    


# for iu in range(0,nmv):
#     ax2=fig2.add_subplot(nmv+nde,1,iu+1)
#     ax2.plot(xi,Uk[iu,:].T, label='Medição')
#     ax2.plot([1,nsim],[umin[iu], umin[iu]],'--r')
#     ax2.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
#     ax2.set_ylabel(label[iu])
#     ax2.set(xlim=(xi[0], nsim*ts))
#     if iu==0:
#         ax2.set(ylim=(30, 70))

#     plt.grid(True)


In [ ]:
fig3=plt.figure()
label = ['Pbh (bar)','Pwh (bar)','q(m3/s)'];
for iu in range(0,3):
    ax3=fig3.add_subplot(3,1,iu+1)
    if iu==2:
        ax3.plot(t,(Xk[iu,:].T)*3600, label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)
    else:
        ax3.plot(t,Xk[iu,:].T/1e5, label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)


In [ ]:
# Xk[2,:]=Xk[2,:]*xc[2] #desnormalizar vazão
# Yk[1,:]=Yk[1,:]*Hc #desnormalizar Head

In [ ]:
exec(compile(open('envelope.py', "rb").read(), 'envelope.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
fig4,ax4=plt.subplots()
plt.grid(True)
BCS['Envelope']['fig'](ax4); # grafico do envelope
#
# Evolução dentro do envelope
ax4.plot(Xk[2,0:].T*3600,Yk[1,0:].T,'--k')
ax4.plot(Xk[2,0]*3600,Yk[1,0],'o')#,'MarkerFaceColor',[0,1,0],'MarkerEdgeColor',[0,0,0])
ax4.plot(Xk[2,-1]*3600,Yk[1,-1],'o')#,'MarkerFaceColor',[1,0,0],'MarkerEdgeColor',[0,0,0])
ax4.annotate('t=0',
             xy=(float(Xk[2,0]*3600),float(Yk[1,0])),
             xytext=(float(Xk[2,0]*3600)-5,float(Yk[1,0])+10),
             arrowprops=dict(facecolor='green', shrink=0.01))

ax4.annotate(f't={nsim}',
             xy=(float(Xk[2,-1]*3600),float(Yk[1,-1])),
             xytext=(float(Xk[2,-1]*3600)-7,float(Yk[1,-1])+10),
             arrowprops=dict(facecolor='red', shrink=0.01))
plt.show()

Uk[2:,:]=Uk[2:,:]*1e5

Envelope carregado


KeyError: 'Envelope'

In [ ]:
#np.savez('BCS_data_train_limitado_f_zc_pm_pr_ident.npz', t=xi, x1=Xk[0,:].T,x2=Xk[1,:].T,x3=Xk[2,:].T,U=Uk.T)